In [114]:
import pandas as pd
import glob
import numpy as np
import json
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, f1_score

## fastText predictions

In [3]:
FTdf = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/fastText/test_data_yTrue_yPred.json", orient='records')

In [4]:
FTdf.head(1)

,label,article,y_pred
0,0,Buy silver and xrp just a opinion. Been diggin...,2


In [5]:
FTdf.shape

(1326, 3)

In [6]:
FTdf['label'].value_counts()

0    719
2    363
3    119
5     52
4     38
1     35
Name: label, dtype: int64

## GPT2 Epoch 5 predictions

In [16]:
G5df = pd.read_json("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/DataModelsResults/Results/OpenAIGPT2Epochs5/test_data_yTrue_yPred.json", orient='records')

In [17]:
G5df.head(1)

,text,label,y_pred
0,Buy silver and xrp just a opinion. Been diggin...,0,2


In [20]:
G5df.shape

(1326, 3)

In [21]:
G5df['label'].value_counts()

0    719
2    363
3    119
5     52
4     38
1     35
Name: label, dtype: int64

# Sample 30 from each predicted class to be Annotated by Lizzy

In [13]:
FTdf['y_pred'].value_counts()

0    722
2    408
3     94
4     48
5     30
1     24
Name: y_pred, dtype: int64

In [14]:
G5df['y_pred'].value_counts()

0    825
2    389
3     76
5     26
4      7
1      3
Name: y_pred, dtype: int64

In [15]:
df = FTdf.copy()

In [21]:
sampled_df = pd.concat([
    df[df['y_pred'] == i].sample(n=min(len(df[df['y_pred'] == i]), 30)) for i in range(0, 6)
])

In [22]:
sampled_df.shape

(174, 3)

In [23]:
sampled_df = sampled_df.sample(frac=1)

In [24]:
sampled_df.head(10)

,label,article,y_pred
455,3,"""The principal instructed the teacher to leave...",1
379,2,Enough is enough already! These hollywood nut...,3
571,0,The only people interested in this January 6th...,2
1196,3,The liberals are running their mouths so much ...,0
644,0,Sorry nothing to see here Why was there only o...,1
1047,2,They are not prop weapons but real live weapon...,2
1164,2,Unless God decides to intervene for a nation w...,2
1243,2,"By the way, is there a US legal code or act th...",4
620,3,"He's been a mean, POS for a very long time. I'...",1
1097,0,Sorry. The Pentagon looks like a “target” from...,0


In [25]:
sampled_df.to_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples174ForAnnotation.csv")

# Get GPT-2 predictions on 174

In [84]:
sampled_df=pd.read_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples174ForAnnotation.csv")

In [85]:
sampled_df.head(1)

,Unnamed: 0,label,article,y_pred
0,455,3,"""The principal instructed the teacher to leave...",1


In [86]:
sampled_df=sampled_df.rename(columns={'y_pred':'fastText','article':'reply'})

In [87]:
# # Find the values of LizTest174['reply'] in G5df['reply']
# matching_replies = G5df[G5df['text'].isin(sampled_df['reply'])]
# # Extract G5df['y_pred'] for the matching replies
# sampled_df['GPT2Epoch5'] = matching_replies['y_pred']

In [88]:
# Create a new column in LizTest174 to store the label values
sampled_df['GPT2Epoch5'] = None

# Iterate over each row in LizTest174
for index, row in sampled_df.iterrows():
    reply = row['reply']
    
    # Check if the reply is in G5df['reply']
    matching_row = G5df[G5df['text'] == reply]
    
    # If a match is found, retrieve the label value from G5df and fill it in LizTest174
    if not matching_row.empty:
        label = matching_row.iloc[0]['y_pred']
        sampled_df.at[index, 'GPT2Epoch5'] = label

In [90]:
sampled_df.shape

(174, 5)

In [91]:
sampled_df.head(1)

,Unnamed: 0,label,reply,fastText,GPT2Epoch5
0,455,3,"""The principal instructed the teacher to leave...",1,0


In [92]:
sampled_df.to_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples174ForAnnotationwithfastTextGPT2Epoch5.csv")

# Annoation by Lizzy on Test set

In [93]:
Liz174=pd.read_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples174ForAnnotationByLizzy.csv")

In [94]:
Liz174.head(1)

,#1,#,reply,Label,Label1 extra,Label2 extra,Label3 extra
0,1,455,"""The principal instructed the teacher to leave...",1,NaN,NaN,NaN


In [95]:
Liz174['Label'].value_counts()

6    56
4    28
1    25
5    25
3    23
2    17
Name: Label, dtype: int64

In [96]:
Test174=pd.read_csv("/home/ravi/UCF Dropbox/KAMALAKKANNAN RAVI/guyonDesktop/DATA_AutomatedHarmDetection/Annotate/TestSamples174ForAnnotationwithfastTextGPT2Epoch5.csv")

In [101]:
Test174.head(1)

,Unnamed: 0,Unnamed: 0.1,label,reply,fastText,GPT2Epoch5
0,0,455,3,"""The principal instructed the teacher to leave...",1,0


In [103]:
Test174['label'].value_counts()

0    64
2    36
5    22
4    21
3    20
1    11
Name: label, dtype: int64

## Custom Weighted Kappa

In [104]:
Liz174['Label'] = Liz174['Label'].replace([6], 0)

In [105]:
Liz174['Label'].value_counts()

0    56
4    28
1    25
5    25
3    23
2    17
Name: Label, dtype: int64

In [106]:
# Define the weight matrix
weightsLR = np.array([
    [0, 0.2, 0.2, 0.3, 0.5, 0.5],
    [0.2, 0, 0.1, 0.2, 0.4, 0.4],
    [0.2, 0.1, 0, 0.2, 0.4, 0.4],
    [0.3, 0.2, 0.2, 0, 0.2, 0.1],
    [0.5, 0.4, 0.4, 0.2, 0, 0.3],
    [0.5, 0.4, 0.4, 0.2, 0.3, 0]
])

def customWeightedKappa(y1, y2, weights):
    # Calculate the confusion matrix
    confusion = confusion_matrix(y1, y2)

    # Normalize the confusion matrix to represent probabilities
    total_ratings = np.sum(confusion)
    confusion = confusion / total_ratings

    # Compute the expected probability matrix under the assumption of independence
    ratings_probs = np.sum(confusion, axis=1)
    expected = np.outer(ratings_probs, ratings_probs)

    # Compute the weighted Kappa score
    kappa = 1 - np.sum(weights * confusion) / np.sum(weights * expected)
    
    return kappa

In [107]:
print(customWeightedKappa(Liz174['Label'], Test174['fastText'], weights=weightsLR))

0.33609116252778193


In [108]:
print(customWeightedKappa(Liz174['Label'], Test174['GPT2Epoch5'], weights=weightsLR))

0.37632806176852196


In [109]:
print(customWeightedKappa(Liz174['Label'], Test174['label'], weights=weightsLR))

0.5082156759465049


In [117]:
print(customWeightedKappa(Test174['fastText'], Test174['GPT2Epoch5'], weights=weightsLR))

0.2063643595863166
